(C) 2023-24 Manfred Schlaegl <manfred.schlaegl@jku.at>, Institute for Complex Systems, JKU Linz

SPDX-License-Identifier: BSD 3-clause "New" or "Revised" License

In [ ]:
import os
import rvvts

## Config

In [ ]:
%run config_internal.ipynb
%run config_host.ipynb

In [ ]:
# MAIN SETTINGS
instance = 100

dut = "VP"
#dut = "QEMU"

#xlen = 32
xlen = 64

#vector_vlen = 128
vector_vlen = 512

testset_type = "ValidSeq"           # pure positive testing
#testset_type = "InvalidValidSeq"   # positive/negative testing


# build config
config = dict()
config.update(config_internal.copy())
config.update(config_host.copy())
config.update(dict(
    dir = os.getcwd() + "/run/TestsetCodeErrMinRunner_" + str(instance) + "_" + dut,

    ArchiveRunner_dut = rvvts.TestsetCodeErrMinRunner,

    # pre-generated test sets
    testset_dir = os.getcwd() + "/TestSets/TestSet_" + testset_type + "_RV" + str(xlen) + "_3MiB_RVV_VLEN_" + str(512) + "_100",
    #testset_dir = os.getcwd() + "/tests_no_error_instr/CovGuidedTestset_VP_RV" + str(xlen) + "_RVV_1000_10.0",
    #testset_dir = os.getcwd() + "/run/CovGuidedTestset_RV" + str(xlen) + "_RVV_test1_100_10.0",
    testset_pattern = "*.json",    
    testset_max_fragments_per_run = 10,
    
    debug_port = 8000 + instance,

    CompareRunner_dut = rvvts.DuTGDBRunner,

    # Optional: Coverage
    #RefCovRunner_coverage = rvvts.RISCVOVPSIMCoverageRunner,
    #RISCVOVPSIMCover_metric = "basic", # basic, extended or mnemonic
    #RISCVOVPSIMCover_metric = "extended", # basic, extended or mnemonic
    #RISCVOVPSIMCover_sum_enable = True,
    
    xlen = xlen,
    vector_vlen = vector_vlen,
))

if dut == "VP":
    config["DuTGDBRunner_dut"] = rvvts.VPRunner
elif dut == "QEMU":
    config["DuTGDBRunner_dut"] = rvvts.QEMURunner
else:
    raise Exception("invalid dut")

## Setup

In [ ]:
runner = rvvts.ArchiveRunner(config)

## Replay Testset

In [ ]:
def TestsetCodeErrMin_stats(runner):
    tcem = runner.ArchiveRunner_dut
    cemr = tcem.codeerrminrunner
    covr = cemr.codecomparerunner.compare_runner.CompareRunner_refcov.RefCovRunner_cov

    out = ", SUM: [ test: " + str(tcem.testset_idx + 1) + "/"  + str(tcem.testset_len) + \
            ", subrun: " + str(tcem.subrun) + "/"  + str(tcem.subruns) + \
            ", tests: " + str(cemr.tests) +\
            ", completes: " + str(cemr.completes) +\
            ", ignores: " + str(cemr.ignores) +\
            ", timeouts: " + str(cemr.timeouts) +\
            ", unknown_faults: " + str(cemr.unknown_faults) +\
            ", errors: " + str(cemr.errors) +\
            ", reductions: " + str(cemr.reductions) +\
            ", minimizations: " + str(cemr.minimizations) +\
            ", instr_errors: " + str(cemr.instr_errors)

    if covr and covr.coverage:
        coverage_type = covr.coverage["sum"]["coverage"]["type"]
        coverage = covr.coverage["sum"]["coverage"]["percent"]
        out += ", coverage (" + str(coverage_type) + "): " + str(coverage) + "%"

    out += " ] "
    return out

In [ ]:
stats = runner.ArchiveRunner_dut.get_testset_stats()
stats

In [ ]:
rvvts.runner_bench(
    runner,
    custom_stat_f = TestsetCodeErrMin_stats,
    iter = -1, # inf
    timeout = 3.0,
    stop_on_ignore = True,
)